In [2]:
import os
import shutil
import numpy as np
import tensorflow as tf
import pandas as pd
#import tensorflow_hub as hub
#import tensorflow_text as text

print("TF Version: ", tf.__version__)

TF Version:  2.16.1


In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_loc = "C:\\Users\\mgf-l\\Desktop\\structured_data.xlsx"
df = pd.read_excel(data_loc)
    
df = df.dropna(subset=['category', 'title'])

X = df['title']   
y = df['category']  

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)  


In [4]:
BUFFER_SIZE = 1400
BATCH_SIZE = 64


In [5]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_X, train_y))
test_dataset = tf.data.Dataset.from_tensor_slices((test_X, test_y))


In [6]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b'Pub kronan'
label:  b'Pub'


In [7]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [8]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'Library' b'Libraryet' b'Kansli']

labels:  [b'Other' b'Other' b'Reception hours']


In [9]:
for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])

texts:  [b'S\xc3\xa5ngboks Gasque 04 sl\xc3\xa4pp' b'Caf\xc3\xa9 Bakfickan '
 b'Bistro']

labels:  [b'Club' b'Fika' b'Pub']


In [10]:
VOCAB_SIZE = 10000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda x, y: x))


In [11]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'café', 'pub', 'lunch', 'bakfickan', 'at', 'majs',
       'library', 'breakfast', 'reception', '1q', 'orvars', 'krog',
       'ebbas', 'mottagningstider', 'puben', 'hattstugan', 'ghubben',
       'svantes'], dtype='<U26')

In [12]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[122, 116, 120,  72,   0,   0],
       [  2,   5,   0,   0,   0,   0],
       [ 35,   0,   0,   0,   0,   0]], dtype=int64)

In [13]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'S\xc3\xa5ngboks Gasque 04 sl\xc3\xa4pp'
Round-trip:  sångboks gasque 04 släpp  

Original:  b'Caf\xc3\xa9 Bakfickan '
Round-trip:  café bakfickan    

Original:  b'Bistro'
Round-trip:  bistro     



In [14]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [15]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [16]:
sample_text = ("Majs cafe")

predictions = model.predict(np.array([sample_text]))
print(predictions[0])

ValueError: Invalid dtype: str288

In [ ]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
# Check the content of the datasets
print("Training set:")
for features, labels in train_dataset.take(1):
    print(features, labels)

print("Test set:")
for features, labels in test_dataset.take(1):
    print(features, labels)


In [ ]:
history = model.fit(train_dataset, epochs=10,
                    validation_data=test_dataset,
                    validation_steps=10)